# Part A: load Data

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

In [2]:
data = pd.read_csv('../data/BTCUSDT.csv', sep='|', nrows=1000, header=None, usecols=[0, 1, 2, 3, 4, 5], names=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
print(data.head())

    timestamp     open     high      low    close    volume
0  1502942400  4261.48  4261.48  4261.48  4261.48  1.775183
1  1502942460  4261.48  4261.48  4261.48  4261.48  0.000000
2  1502942520  4280.56  4280.56  4280.56  4280.56  0.261074
3  1502942580  4261.48  4261.48  4261.48  4261.48  0.012008
4  1502942640  4261.48  4261.48  4261.48  4261.48  0.140796


In [3]:
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')
data.set_index('timestamp', inplace=True)
print(data)

                        open     high      low    close    volume
timestamp                                                        
2017-08-17 04:00:00  4261.48  4261.48  4261.48  4261.48  1.775183
2017-08-17 04:01:00  4261.48  4261.48  4261.48  4261.48  0.000000
2017-08-17 04:02:00  4280.56  4280.56  4280.56  4280.56  0.261074
2017-08-17 04:03:00  4261.48  4261.48  4261.48  4261.48  0.012008
2017-08-17 04:04:00  4261.48  4261.48  4261.48  4261.48  0.140796
...                      ...      ...      ...      ...       ...
2017-08-17 20:35:00  4302.96  4302.96  4302.96  4302.96  0.052740
2017-08-17 20:36:00  4302.96  4302.96  4302.96  4302.96  0.585532
2017-08-17 20:37:00  4301.91  4301.91  4301.91  4301.91  0.445848
2017-08-17 20:38:00  4301.91  4301.91  4301.91  4301.91  0.372403
2017-08-17 20:39:00  4301.91  4303.18  4262.53  4303.18  1.161897

[1000 rows x 5 columns]


In [4]:
data_tensor = torch.from_numpy(data.values).to(torch.float32)
print(data_tensor)

tensor([[4.2615e+03, 4.2615e+03, 4.2615e+03, 4.2615e+03, 1.7752e+00],
        [4.2615e+03, 4.2615e+03, 4.2615e+03, 4.2615e+03, 0.0000e+00],
        [4.2806e+03, 4.2806e+03, 4.2806e+03, 4.2806e+03, 2.6107e-01],
        ...,
        [4.3019e+03, 4.3019e+03, 4.3019e+03, 4.3019e+03, 4.4585e-01],
        [4.3019e+03, 4.3019e+03, 4.3019e+03, 4.3019e+03, 3.7240e-01],
        [4.3019e+03, 4.3032e+03, 4.2625e+03, 4.3032e+03, 1.1619e+00]])


In [5]:
data_tensor.dtype, data_tensor.shape, data_tensor.device

(torch.float32, torch.Size([1000, 5]), device(type='cpu'))

# Part B: Computational Graph Internals

In [6]:
x = data_tensor[100].reshape(1, -1)
w = torch.randn_like(x, requires_grad=True)
b = torch.randn(1, requires_grad=True)
x.shape, w.shape, b.shape

(torch.Size([1, 5]), torch.Size([1, 5]), torch.Size([1]))

In [7]:
x, w, b

(tensor([[4291.3701, 4291.3701, 4291.3701, 4291.3701,    0.0000]]),
 tensor([[ 0.5556,  0.4411, -0.6088, -1.0542, -0.0617]], requires_grad=True),
 tensor([0.4099], requires_grad=True))

In [8]:
y = x @ w.T + b
y

tensor([[-2859.0393]], grad_fn=<AddBackward0>)

In [9]:
y.backward()
w.grad, b.grad

(tensor([[4291.3701, 4291.3701, 4291.3701, 4291.3701,    0.0000]]),
 tensor([1.]))

In [10]:
x, w, b

(tensor([[4291.3701, 4291.3701, 4291.3701, 4291.3701,    0.0000]]),
 tensor([[ 0.5556,  0.4411, -0.6088, -1.0542, -0.0617]], requires_grad=True),
 tensor([0.4099], requires_grad=True))

# Part C: Gradient Accumulation

In [11]:
w = torch.randn_like(x, requires_grad=True)
b = torch.randn(1, requires_grad=True)

for i in range(3):
    y = x @ w.T + b
    y.backward()
    
    print(w.grad, b.grad, '\n')

tensor([[4291.3701, 4291.3701, 4291.3701, 4291.3701,    0.0000]]) tensor([1.]) 

tensor([[8582.7402, 8582.7402, 8582.7402, 8582.7402,    0.0000]]) tensor([2.]) 

tensor([[12874.1104, 12874.1104, 12874.1104, 12874.1104,     0.0000]]) tensor([3.]) 



In [12]:
w = torch.randn_like(x, requires_grad=True)
b = torch.randn(1, requires_grad=True)

for i in range(3):
    y = x @ w.T + b
    y.backward()

    print(w.grad, b.grad, '\n')
    
    w.grad.zero_()
    b.grad.zero_()

tensor([[4291.3701, 4291.3701, 4291.3701, 4291.3701,    0.0000]]) tensor([1.]) 

tensor([[4291.3701, 4291.3701, 4291.3701, 4291.3701,    0.0000]]) tensor([1.]) 

tensor([[4291.3701, 4291.3701, 4291.3701, 4291.3701,    0.0000]]) tensor([1.]) 



# Part D: Config-driven Architecture

In [13]:
import yaml
import pandas as pd

with open('../configs/configs.yaml', 'r') as file:
    config = yaml.safe_load(file)

print("Loaded Config Dictionary:")
print(config)
print("-" * 30)

file_path = config['data']['file_path']
sep = config['data']['separator']
nrows = config['data']['train_rows']
cols = config['data']['features']

data = pd.read_csv(
    file_path, 
    sep=sep, 
    nrows=nrows, 
    header=None, 
    usecols=[0, 1, 2, 3, 4, 5], 
    names=cols
)

print(data.head())

Loaded Config Dictionary:
{'data': {'file_path': '../data/BTCUSDT.csv', 'separator': '|', 'train_rows': 1000, 'features': ['timestamp', 'open', 'high', 'low', 'close', 'volume']}}
------------------------------
    timestamp     open     high      low    close    volume
0  1502942400  4261.48  4261.48  4261.48  4261.48  1.775183
1  1502942460  4261.48  4261.48  4261.48  4261.48  0.000000
2  1502942520  4280.56  4280.56  4280.56  4280.56  0.261074
3  1502942580  4261.48  4261.48  4261.48  4261.48  0.012008
4  1502942640  4261.48  4261.48  4261.48  4261.48  0.140796


# Part E: retain_graph

In [14]:
def custom_trading_loss(pnl):
    if pnl > 0:
        loss = pnl * 0.05
    elif pnl < 0:
        loss = pnl ** 2
    return loss

In [15]:
pnl1 = torch.tensor([4.0], requires_grad=True)

y1 = custom_trading_loss(pnl1)
y1.backward()
pnl1.grad

tensor([0.0500])

In [16]:
pnl2 = torch.tensor([-3.0], requires_grad=True)

y2 = custom_trading_loss(pnl2)
y2.backward()
pnl2.grad

tensor([-6.])

In [17]:
y2.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [18]:
pnl2 = torch.tensor([-3.0], requires_grad=True)

y2 = custom_trading_loss(pnl2)
y2.backward(retain_graph=True)
pnl2.grad

tensor([-6.])

In [19]:
y2.backward()
pnl2.grad

tensor([-12.])